In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import allantools
from runData import runData

In [ ]:
pd.set_option("display.precision", 15)      # Show up to 15 decimal places
#baseName = 'fixedL1l53'
#baseName = 'baseline1'
dirName  = 'labData/'
#rowLimit = 2000 # Rows to keep after joining ticc and timTp samples

In [ ]:
# Big Picture
#   Read TICC data from runs of interest
#   Process it, do simple analysis without timTp data
#   Read timTp data
#   Join with TICC data and analyze
#   Plot results

In [ ]:
# Read TAPR TICC data into a dataframe, as captured to a file by ticc.py running on a host.
# Events are rising edges of a PPS signal from a DUT, resulting in a timestamp on the TICC's reference clock.
# Columns:
#  ppsHostClock: Host clock when serial data for PPS event timestamp was read (ticc.py logs this in UTC)
#  ppsRefClock:  Reference clock when PPS event happened (elapsed seconds since TICC started)
#
# The frequency of the TICC reference clock comes from an external 10 MHz source, a Geppetto Electronics GNSSDO in my case.
# It should be almost exactly 1e7 times the PPS frequency, with an arbitray phase relationship.
# So we expect the whole number portion of ppsRefClock to increment by 1 every second, while the fractional seconds jitter around
# some slowly-changing phase offset.
# Therefore, there's very little information in the whole seconds, while the fractional seconds contain the most interesting data.
# And as the whole number grows with a floating point representation, precision is lost in the fractional digits.
# So once we confirm the whole number of seconds is behaving as expected, we can drop it and focus on the fractional seconds.
# Whatever slowly-changing phase offset exists, it won't impact the deviation metrics.
# We treat the fractional seconds as an instantaneous (but nosiy) measurement of the phase error against the ref clock.
#
# There is a very small, but non-zero chance that the static phase offset plus the jitter causes sequential PPS timestamps to be
# within the same second or more than one second apart, leading to missing or duplicate whole seconds.
# Instead of properly handling whole seconds when this happens, just fail on assertions.
def readTicc(baseName, chan):
    ticcFile = f"{dirName}/{baseName}.ticc{chan}.csv"
    ticcData = pd.read_csv(ticcFile, dtype={'ppsHostClock': str, 'ppsRefClock': str})

    # Convert host timestamp string to UTC timestamp
    ticcData["ppsHostClock"] = pd.to_datetime(ticcData.ppsHostClock, utc=True)

    # Assuming host clock sync is better than serialization latency of timestamp arriving,
    # floor of host clock second will be the navigation epoch sencond.
    # Will be used for later join with TIM-TP timestamps.
    ticcData["epochSec"] = ticcData["ppsHostClock"].dt.floor("s")


    # Split ppsRefClock string into whole and fractional seconds
    ticcData[["rcWhole", "rcFrac"]] = ticcData.ppsRefClock.str.split(".", n=1, expand=True)

    # Check for missing or duplicate whole seconds
    ticcData['rcWhole'] = ticcData['rcWhole'].astype(int)
    expected = set(range(ticcData.rcWhole.min(), ticcData.rcWhole.max() + 1))
    observed = set(ticcData.rcWhole)
    missing = sorted(expected - observed)
    duplicates = ticcData.rcWhole[ticcData.rcWhole.duplicated()].unique().tolist()
    assert len(missing)    == 0, f"Missing whole seconds in ticc{chan}: {missing}"
    assert len(duplicates) == 0, f"Duplicate whole seconds in ticc{chan}: {duplicates}"

    # Convert ref clock fractional part from digit string to float
    ticcData['rcFrac'] = "0." + ticcData['rcFrac'].astype(str)
    ticcData['rcFrac'] = ticcData['rcFrac'].astype(float)

    # Also get fractional part of host clock
    ticcData['hcFrac'] = (ticcData.ppsHostClock.astype('int64')-1e9*(ticcData.ppsHostClock.astype('int64')//1e9))/1e9

    # With overly careful consideration of maintining floating point precision, get interval between PPS events on ref clock.
    ticcData['rcTi'] = (ticcData.rcWhole-ticcData.rcWhole.shift(1)) + (ticcData.rcFrac - ticcData.rcFrac.shift(1)) # Time interval between refClock samples on ref clock

    ticcData['bn'  ] = baseName
    ticcData['dut' ] = runData[baseName][chan]
    ticcData['chan'] = chan
    return ticcData

In [ ]:
dfs = []
dfs.append(readTicc('fixedL1l52', 'A'))
dfs.append(readTicc('fixedL1l52', 'B'))
dfs.append(readTicc('fixedL1l53', 'A'))
dfs.append(readTicc('fixedL1l53', 'B'))
ticc = pd.concat(dfs, ignore_index=True)

In [ ]:
ticc.dtypes

In [ ]:
ticc
# XXX next step: define function to generate statistics from selected rows of ticc, plot, and label them

In [ ]:
# Bring data from TICC channels for each run together for each ref clock second.
ticcA = ticc[ticc.chan == 'A'][['bn', 'rcWhole', 'rcFrac', 'epochSec']]
ticcB = ticc[ticc.chan == 'B'][['bn', 'rcWhole', 'rcFrac'            ]]

# Perform inner join on bn and rcWhole
rcSec = pd.merge(ticcA, ticcB, on=['bn', 'rcWhole'], suffixes=('A', 'B'))

# Derive columns of interest from the TICC data.
rcSec['rcFracAB'] = rcSec.rcFracA - rcSec.rcFracB  # Phase difference between the two channels

In [ ]:
rcSec

In [ ]:
for bn in rcSec.bn.unique():
    print(f"Stats for run: {bn}")
    print(rcSec[rcSec.bn==bn].rcFracAB.describe()[1:]*1e9)

In [ ]:
beg = 1000
end = 2000
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(rcSec[rcSec.bn==bn].epochSec[beg:end], rcSec[rcSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.title(f"Phase Difference (rcFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (rcFracAB)')
plt.grid()
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(22, 6))
fig.canvas.draw()
bbox = ax.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
widthPx = int(bbox.width*fig.dpi)
print("Figure width in pixels:", widthPx)
print(fig.get_figwidth())
plt.hist(1e9*(ticc[ticc.bn==bn].rcTi[beg:end]-1.0), bins=widthPx, color='blue', alpha=0.7)
plt.title('Histogram of Time Error Between PPS Pulses')
plt.xlabel('Time Error (ns)')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.gca().xaxis.set_major_formatter(FuncFormatter(lambda y, _: f"{y:g} ns"))
plt.show()



In [ ]:
def readTimTp(baseName, chan):
    timTp = pd.read_csv(f"{dirName}/{baseName}.timTp{chan}.csv")

    # Confirm expected values in constant columns, then drop them
    assert (timTp['timeBase'   ] ==  1).all(), "Not all rows in timTp.timeBase are equal to 1"
    assert (timTp['utc'        ] ==  1).all(), "Not all rows in timTp.utc are equal to 1"
    assert (timTp['raim'       ] ==  2).all(), "Not all rows in timTp.raim are equal to 2"
    assert (timTp['qErrInvalid'] ==  0).all(), "Not all rows in timTp.qErrInvalid are equal to 0"
    assert (timTp['TpNotLocked'] ==  0).all(), "Not all rows in timTp.TpNotLocked are equal to 0"
    assert (timTp['timeRefGnss'] == 15).all(), "Not all rows in timTp.timeRefGnss are equal to 15"
    assert (timTp['utcStandard'] ==  3).all(), "Not all rows in timTp.utcStandard are equal to  3"
    assert (timTp['towSubMS'   ] ==  0).all(), "Not all rows in timTp.towSubMS are equal to  0"
    timTp.drop(columns=['timeBase', 'utc', 'raim', 'qErrInvalid', 'TpNotLocked', 'timeRefGnss', 'utcStandard', 'towSubMS'], inplace=True)

    # Constants for time conversion
    gps_epoch = pd.Timestamp("1980-01-06 00:00:00", tz="UTC")
    leap_seconds = pd.Timedelta(seconds=18)  # current GPS-UTC offset (2025)

    # Vectorized conversion from GPS week and TOW to epoch seconds
    timTp["epochSec"] = (
        gps_epoch
        + pd.to_timedelta(timTp.week  * 7, unit="D" )
        + pd.to_timedelta(timTp.towMS    , unit="ms")
    )
    timTp.drop(columns=['week', 'towMS'], inplace=True)

    timTp['qErrFrac'] = timTp.qErr/1e12 # Convert qErr from picoseconds to seconds

    timTp['bn'] = baseName
    timTp['dut' ] = runData[baseName][chan]
    timTp['chan'] = chan
    return timTp

In [ ]:
dfs = []
dfs.append(readTimTp('fixedL1l52', 'A'))
dfs.append(readTimTp('fixedL1l52', 'B'))
dfs.append(readTimTp('fixedL1l53', 'A'))
dfs.append(readTimTp('fixedL1l53', 'B'))
timTp = pd.concat(dfs, ignore_index=True)

In [ ]:
timTp

In [ ]:
# Bring data from TIM-TP messages for each run together for each epoch second.
timTpA = timTp[timTp.chan == 'A'][['bn', 'epochSec', 'qErr', 'qErrFrac']]
timTpB = timTp[timTp.chan == 'B'][['bn', 'epochSec', 'qErr', 'qErrFrac']]

# Perform inner join on bn and epoch second
epSec = pd.merge(timTpA, timTpB, on=['bn', 'epochSec'], suffixes=('A', 'B'))

In [ ]:
# Merge TICC data from above with TIM-TP data on epoch second.
epSec = pd.merge(epSec, rcSec, on=['bn', 'epochSec'], how="inner")

In [ ]:
epSec

In [ ]:
# Correct rcFrac with qErr
epSec['rcFracCorrA'] = epSec.rcFracA+epSec.qErrFracA
epSec['rcFracCorrB'] = epSec.rcFracB+epSec.qErrFracB

# Corrected phase difference between the two channels
epSec['rcFracCorrAB'] = epSec.rcFracCorrA - epSec.rcFracCorrB

In [ ]:
epSec

In [ ]:
for bn in epSec.bn.unique():
    print(f"Stats for run: {bn}")
    print(epSec[epSec.bn==bn][:1000].rcFracCorrAB.describe()[1:]*1e9)

In [ ]:
# If I have two series of PPS timestamps from imperfect clocks and I want to compare how well they track each other, paying less attention to the absolute phase error and more to the relative phase error, are there statistical analysis methods like Allan deviation that I can apply to the difference between clocks?

In [ ]:
beg = 11000
end = 11500
bn = 'fixedL1l53'
plt.figure(figsize=(24, 6))
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracAB[beg:end]*1e9, marker='.', linestyle='-', color='b')
plt.plot(epSec[epSec.bn==bn].epochSec[beg:end], epSec[epSec.bn==bn].rcFracCorrAB[beg:end]*1e9, marker='.', linestyle='-', color='r')
plt.title(f"Phase Difference (epFracAB) vs Epoch Second for Run {bn}")
plt.xlabel('Epoch Second')
plt.ylabel('Phase Difference (epFracAB)')
plt.grid()
plt.show()
